## Importing libraries

In [1]:
import random
import pandas as pd
import random
import os
import json
import re
from IPython import get_ipython

## Extracting Strategies

In [2]:
# Function to extract the single function from a given IPYNB file and get its name
def extract_function_from_ipynb(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook_content = json.load(f)
    
    function_code = ""
    function_name = None
    function_name_pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    
    for cell in notebook_content['cells']:
        if cell['cell_type'] == 'code':
            code = ''.join(cell['source'])
            function_code += code + '\n'
            match = function_name_pattern.search(code)
            if match:
                function_name = match.group(1)
    
    return function_code, function_name

# Function to run the extracted functions
def run_extracted_functions(function_code):
    ipython = get_ipython()
    ipython.run_cell(function_code)

# Path to the folder containing IPYNB files
folder_path = 'Anonymized_Strategies'

all_function_code = ""
strategies_Name = []

# Iterate through all IPYNB files in the specified folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.ipynb'):
        file_path = os.path.join(folder_path, file_name)
        function_code, function_name = extract_function_from_ipynb(file_path)
        all_function_code += function_code + '\n'
        if function_name:
            strategies_Name.append(function_name)

# Run all the extracted function code
run_extracted_functions(all_function_code)

# Print the list of function names
print("Extracted function names (strategies):", strategies_Name)

Extracted function names (strategies): ['Student_001', 'Student_002', 'Student_003', 'Student_004', 'Student_005', 'Student_006', 'Student_007', 'Student_008', 'Student_009', 'Student_010', 'Student_011', 'Student_012', 'Student_013', 'Student_014']


In [5]:
# changing the strings to function names
strategies = [globals()[name] for name in strategies_Name]
strategies

[<function __main__.Student_001(history)>,
 <function __main__.Student_002(history)>,
 <function __main__.Student_003(history)>,
 <function __main__.Student_004(history)>,
 <function __main__.Student_005(history)>,
 <function __main__.Student_006(history)>,
 <function __main__.Student_007(history)>,
 <function __main__.Student_008(history)>,
 <function __main__.Student_009(history)>,
 <function __main__.Student_010(history)>,
 <function __main__.Student_011(history)>,
 <function __main__.Student_012(history)>,
 <function __main__.Student_013(history)>,
 <function __main__.Student_014(history)>]

In [6]:
print("Number of players(Strategies):", len(strategies))

Number of players(Strategies): 14


An altered version of play function for gathering information in competition

In [7]:
def play_game(strategy1, strategy2):

    score1 = 0
    score2 = 0

    for _ in range(20):
        move1 = strategy1(history)
        flipped_history = [(move2, move1) for move1, move2 in history]
        move2 = strategy2(flipped_history)

        if move1 == "Cooperate" and move2 == "Cooperate":
            score1 += 3
            score2 += 3
        elif move1 == "Cooperate" and move2 == "Defect":
            score1 += 0
            score2 += 5
        elif move1 == "Defect" and move2 == "Cooperate":
            score1 += 5
            score2 += 0
        elif move1 == "Defect" and move2 == "Defect":
            score1 += 1
            score2 += 1

        history.append((move1, move2))

    if score1 > score2:
        return strategy1.__name__, strategy2.__name__, score1, score2, strategy1.__name__ + " wins"
    elif score2 > score1:
        return strategy1.__name__, strategy2.__name__, score1, score2, strategy2.__name__ + " wins"
    else:
        return strategy1.__name__, strategy2.__name__, score1, score2, "It's a tie"

# Main competition

Since some strategies are not deterministic, we use a specified random seed to generate numbers, which ensures the results are reproducible.

In [8]:
# Generate a random integer between 0 and 50
random_seed = random.randint(0, 100)
random_seed

10

The random number is 10.

In [9]:
# Using the predefined random seed for reproducibility
random.seed(random_seed)

# Initialize results storage
results = []
histories = []

# Play games between each unique pair of strategies
for i in range(len(strategies)):
    for j in range(i + 1, len(strategies)):
        strategy1 = strategies[i]
        strategy2 = strategies[j]
        history = []
        result = play_game(strategy1, strategy2)
        results.append(result)
        histories.append(history)

# Create DataFrame
df = pd.DataFrame(results, columns=["Strategy 1", "Strategy 2", "Score 1", "Score 2", "Result"])

# Calculate score summations, wins, losses, draws
summary = []
for strategy in strategies:
    name = strategy.__name__
    total_score = df.loc[df["Strategy 1"] == name, "Score 1"].sum() + df.loc[df["Strategy 2"] == name, "Score 2"].sum()
    wins = len(df[(df["Strategy 1"] == name) & (df["Result"] == name + " wins")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] == name + " wins")])
    losses = len(df[(df["Strategy 1"] == name) & (df["Result"] != name + " wins") & (df["Result"] != "It's a tie")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] != name + " wins") & (df["Result"] != "It's a tie")])
    draws = len(df[(df["Strategy 1"] == name) & (df["Result"] == "It's a tie")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] == "It's a tie")])
    
    summary.append((name, total_score, wins, losses, draws))

summary_df = pd.DataFrame(summary, columns=["Strategy", "Total Score", "Wins", "Losses", "Draws"])

summary_df = summary_df.sort_values(by='Total Score', ascending=False)
summary_df

,Strategy,Total Score,Wins,Losses,Draws
2,Student_003,662,0,5,8
0,Student_001,658,6,4,3
6,Student_007,656,10,0,3
5,Student_006,648,10,0,3
13,Student_014,640,0,5,8
7,Student_008,633,0,4,9
12,Student_013,633,1,3,9
10,Student_011,626,6,5,2
3,Student_004,615,0,6,7
1,Student_002,609,0,6,7


In [10]:
df

,Strategy 1,Strategy 2,Score 1,Score 2,Result
0,Student_001,Student_002,82,27,Student_001 wins
1,Student_001,Student_003,44,44,It's a tie
2,Student_001,Student_004,80,30,Student_001 wins
3,Student_001,Student_005,56,41,Student_001 wins
4,Student_001,Student_006,12,52,Student_006 wins
...,...,...,...,...,...
86,Student_011,Student_013,57,57,It's a tie
87,Student_011,Student_014,61,56,Student_011 wins
88,Student_012,Student_013,50,50,It's a tie
89,Student_012,Student_014,50,50,It's a tie


In [11]:
# Saving all games' results
df.to_excel(r'Anonymized_Games.xlsx', index=False)
print("Saving was successful!")

Saving was successful!


In [12]:
# Saving the result table
summary_df.to_excel(r'Anonymized_Result_Table.xlsx', index=False)
print("Saving was successful!")

Saving was successful!


In [13]:
print("Number of all played games: ", len(histories))

Number of all played games:  91


An arbitrary history game can be obtained as follows:

In [14]:
Play_number = 7
histories[Play_number]

[('Defect', 'Defect'),
 ('Cooperate', 'Defect'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Defect', 'Cooperate'),
 ('Cooperate', 'Defect'),
 ('Cooperate', 'Cooperate'),
 ('Defect', 'Cooperate'),
 ('Defect', 'Defect'),
 ('Defect', 'Defect'),
 ('Defect', 'Defect'),
 ('Cooperate', 'Defect'),
 ('Defect', 'Cooperate'),
 ('Defect', 'Defect'),
 ('Cooperate', 'Defect'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Defect', 'Cooperate'),
 ('Defect', 'Defect'),
 ('Defect', 'Defect')]